## Imports and reading in data

In [2]:
#imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier

In [133]:
#Paths to files
data_path = 'Q:/hackers09/shared/data/'
cancer_path = os.path.join(data_path, 'df_cancer.csv')
echo_path = os.path.join(data_path, 'df_echo.csv')
encounter_path = os.path.join(data_path, 'df_encounter.csv')
labs_path = os.path.join(data_path, 'df_labs.csv')
outcome_path = os.path.join(data_path, 'df_outcome.csv')
problist_path = os.path.join(data_path, 'df_problist.csv')
radiology_path = os.path.join(data_path, 'df_radiology.csv')
registry_path = os.path.join(data_path, 'df_registry.csv')
vitals_path = os.path.join(data_path, 'df_vitals.csv')

In [41]:
#Read in dataframes
cancer_df = pd.read_csv(cancer_path, encoding='ISO-8859-1')
cancer_df.set_index("HSP_ENC", inplace = True)
echo_df = pd.read_csv(echo_path, encoding='ISO-8859-1')
echo_df.set_index("HSP_ENC", inplace = True)
encounter_df = pd.read_csv(encounter_path, encoding='ISO-8859-1')
encounter_df.set_index("HSP_ENC", inplace = True)
labs_df = pd.read_csv(labs_path, encoding='ISO-8859-1')
labs_df.set_index("HSP_ENC", inplace = True)
outcome_df = pd.read_csv(outcome_path, encoding='ISO-8859-1')
outcome_df.set_index("HSP_ENC", inplace = True)
problist_df = pd.read_csv(problist_path, encoding='ISO-8859-1')
problist_df.set_index("HSP_ENC", inplace = True)
radiology_df = pd.read_csv(radiology_path, encoding='ISO-8859-1')
radiology_df.set_index("HSP_ENC", inplace = True)
registry_df = pd.read_csv(registry_path, encoding='ISO-8859-1')
registry_df.set_index("HSP_ENC", inplace = True)
vitals_df = pd.read_csv(vitals_path, encoding='ISO-8859-1')
vitals_df.set_index("HSP_ENC", inplace = True)

##  
## Define functions to merge datasets

### Cancer

In [23]:
#function to merge cancer data
def merge_cancer(enc_df, cnc_df):
    #Make modifications to cancer df
    mod_cancer_df = cnc_df.copy()
    mod_cancer_df.reset_index(level=0, inplace=True)
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Missing Remission Date', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Previously Positive', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not Documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'No Cancer', 'cancer_at_enc'] = '1'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown', 'cancer_at_enc'] = '2'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Cancer', 'cancer_at_enc'] = '3'
    mod_cancer_df['cancer_at_enc'] = mod_cancer_df['cancer_at_enc'].astype(int)
    mod_cancer_df.drop_duplicates(['PATIENT_ID', 'HSP_ENC'])
    
    #Take only cancer status column with max value
    drop_cancer_df = mod_cancer_df[['HSP_ENC', 'cancer_at_enc']]
    drop_cancer_df = drop_cancer_df.groupby('HSP_ENC',group_keys=False).apply(lambda x: x.loc[x['cancer_at_enc']==x['cancer_at_enc'].max()])

    #Merge with encounter df and drop dups
    mergeRes = pd.merge(enc_df, drop_cancer_df, on='HSP_ENC', how='left')
    mergeRes = mergeRes.drop_duplicates('HSP_ENC')

    #Replace NaN in encounter df with 0 (Never had cancer)
    mergeRes['cancer_at_enc'].fillna(0, inplace=True)
    
    #Renaming the cancer column
    mergeRes.rename(columns={'cancer_at_enc': 'CANCER_RANK'}, inplace=True)
    
    mergeRes.reset_index(drop=True)
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

### Registry

In [6]:
def merge_registry(enc_df, reg_df):
    reg_list = reg_df.REGISTRY_NAME.unique().tolist()
    reg_df_copy = reg_df.copy()
    reg_df_copy.reset_index(level=0, inplace=True)

    tst_df = reg_df_copy[['HSP_ENC']].copy()
    for item in reg_list:
        tst_df[item] = False
    tst_df = tst_df.drop_duplicates()


    for index, row in reg_df_copy.iterrows():
        enc_id = reg_df_copy.iloc[index, 0]
        curr_reg = reg_df_copy.iloc[index, 2]
        tst_df.loc[tst_df['HSP_ENC'] == enc_id, [curr_reg]] = True

    #Merge with encounter df and drop dups
    mergeRes = pd.merge(enc_df, tst_df, on='HSP_ENC', how='left')
    mergeRes = mergeRes.drop_duplicates('HSP_ENC')
    
    #Replace NaN in encounter df with False, no record
    for item in reg_list:
        mergeRes[item].fillna(False, inplace=True)
    
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

### Vitals

In [7]:
def merge_vitals(enc_df, data_path):
    vital_data_path = os.path.join(data_path, 'vitals.csv')
    vital_data_df = pd.read_csv(vital_data_path, encoding='ISO-8859-1')
    vital_data_df = vital_data_df.drop('Unnamed: 0', 1)
    
    mergeRes = pd.merge(enc_df, vital_data_df, on='HSP_ENC', how='left')
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

In [62]:
def merge_co_morbid(enc_df, data_path):
    co_mobid_path = os.path.join(data_path, 'comorbitidity_score_sm2.csv')
    co_mobid_df = pd.read_csv(co_mobid_path, encoding='ISO-8859-1')
    
    mergeRes = pd.merge(enc_df, co_mobid_df, on='HSP_ENC', how='left')
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

### Labs

In [63]:
def merge_labs(enc_df, data_path):
    lab_data_path = os.path.join(data_path, 'cleaned_condensed_lab_data.csv')
    lab_data_df = pd.read_csv(lab_data_path, encoding='ISO-8859-1')
    
    mergeRes = pd.merge(enc_df, lab_data_df, on='HSP_ENC', how='left')
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

### Radiology / CT / Echo / Outcomes

In [8]:
def merge_encounter_radiology(df_encounter, df_radiology, cutoff_OrderTime=12, cutoff_ED_Disp=12):
    # generate a new df_CT dataframe
    # 1. focus on 'CT ANGIOGRAM' only
    # 2. order time within 12h
    # 3. keep only the first order for the outliers (only one data point that has 2 orders)

    df_CT = df_radiology[df_radiology['NAME'].apply(lambda x: x.startswith('CT AN'))]
    df_CT = df_CT[df_CT['ORDER_TIME_DIFFSEC'] <= cutoff_OrderTime*3600]
    df_CT = df_CT.drop_duplicates('HSP_ENC', keep='first')

    # combine df_encounter and df_CT based on 'HSP_ENC' id
    df_encounter = df_encounter[df_encounter['ED_DISP_TIME_DIFFSEC']<=cutoff_ED_Disp*3600]
    df_enc_CT = pd.merge(df_encounter, df_CT, how='left', on='HSP_ENC')
    df_enc_CT.set_index('HSP_ENC', inplace=True)
    
    return df_enc_CT

In [9]:
def expand_outcome(dfOutcome):
    ## Outcome expansion   
    
    # Drop any outcomes after 48 hr
    dfOutcome = dfOutcome[dfOutcome['ORDER_TIME_DIFFSEC']<48*60*60]
    
    # Create columns for each unique outcome
    lsExpandedOutcomeCols = []
    lsOutcomeColsToRetain = dfOutcome.columns[2:].tolist()
    lsUniqueOutcomes = dfOutcome['name_gen'].value_counts().index.tolist()
    for strOutcome in lsUniqueOutcomes:
        lsExpandedOutcomeCols += [strCol + '_' + strOutcome for strCol in lsOutcomeColsToRetain]

    dfExpandedOutcome = pd.DataFrame(columns=lsExpandedOutcomeCols+['PATIENT_ID'])
    dfExpandedOutcome['HSP_ENC'] = dfOutcome['HSP_ENC'].value_counts().index
    dfExpandedOutcome = dfExpandedOutcome.set_index('HSP_ENC')

    for nEnc in dfOutcome['HSP_ENC'].value_counts().index:
        for nIdx in dfOutcome[dfOutcome['HSP_ENC']==nEnc].index:
            lsTempCols = [strCol + '_' + dfOutcome.at[nIdx, 'name_gen'] for strCol in lsOutcomeColsToRetain]
            lsTempCols.append('PATIENT_ID')
            dfExpandedOutcome.loc[nEnc, lsTempCols] = dfOutcome.loc[nIdx, lsOutcomeColsToRetain+['PATIENT_ID']].values

    # Create boolean column for order time < 48 hrs for any outcome
    lsOrderTimeCols = [strCol for strCol in dfExpandedOutcome.columns if 'ORDER_TIME' in strCol]

    dfExpandedOutcome['b48hr'] = np.zeros(dfExpandedOutcome.shape[0])
    for nEnc in dfExpandedOutcome.index:
        for nVal in dfExpandedOutcome.loc[nEnc, lsOrderTimeCols].values:
            if nVal < 172800:
                dfExpandedOutcome.at[nEnc, 'b48hr'] = 1

    # Column for minimum order time from all outcomes
    dfExpandedOutcome['MinOrderTime'] = dfExpandedOutcome.loc[:, lsOrderTimeCols].min(axis=1)
    dfExpandedOutcome = dfExpandedOutcome.sort_values('MinOrderTime')
    
    return dfExpandedOutcome


def compile_echo(dfEcho):    
    # Keep only 12 hr echos
    dfEcho = dfEcho[dfEcho['ORDER_INST_DIFFSEC'] < 12*60*60]

    # Add all narratives together
    lsUniqueEchoEnc = dfEcho['HSP_ENC'].value_counts().index.tolist()

    lsEchoCols = dfEcho.columns.tolist()
    lsEchoCols.remove('new_line')
    lsEchoCols.remove('NARRATIVE')
    dfCompiledEcho = pd.DataFrame(columns=lsEchoCols)
    dfCompiledEcho['HSP_ENC'] = lsUniqueEchoEnc
    dfCompiledEcho = dfCompiledEcho.set_index('HSP_ENC')
    lsEchoCols.remove('HSP_ENC')

    lsCompiledNarratives = []
    for nEnc in lsUniqueEchoEnc:
        lsUniqueEchoOrderId = dfEcho[dfEcho['HSP_ENC']==nEnc]['ORDER_PROC_ID'].value_counts().index.tolist()
        #if len(lsUniqueEchoOrderId)>1:
            #print(nEnc)
        nFirstEchoOrderId = lsUniqueEchoOrderId[0] # Keep first one only
        
        # Compile echo data
        strCompiled = dfEcho[dfEcho['ORDER_PROC_ID']==nFirstEchoOrderId]['NARRATIVE'].str.cat(sep=' ')
        dfCompiledEcho.loc[nEnc, lsEchoCols] = dfEcho[dfEcho['ORDER_PROC_ID']==nFirstEchoOrderId][lsEchoCols].iloc[0]
        lsCompiledNarratives.append(strCompiled)
    dfCompiledEcho['NARRATIVE_compiled'] = lsCompiledNarratives

    # Keep only 12 hr echos
    dfCompiledEcho = dfCompiledEcho[dfCompiledEcho['ORDER_INST_DIFFSEC'] < 12*60*60]
    
    return dfCompiledEcho

def get_merge_dfs(dfOutcome, dfEcho, dfEncounter):
    dfExpandedOutcome = expand_outcome(dfOutcome)
    dfCompiledEcho = compile_echo(dfEcho)
    dfEncounter = dfEncounter.set_index('HSP_ENC')
    
    dfMerged = dfEncounter.merge(dfExpandedOutcome, how='left', on='HSP_ENC')
    dfMerged = dfMerged.merge(dfCompiledEcho, how='left', on='HSP_ENC')
    
    return dfMerged

In [76]:

def preproc_labeled_echo(dfEchoLabeled):
    # Changing mlid to mild
    dfEchoLabeled['function'] = dfEchoLabeled['function'].replace('mlid', 'mild')
    dfEchoLabeled['dilation'] = dfEchoLabeled['dilation'].replace('mlid', 'mild')
    
    # na to 0, mild to 1, moderate to 2, severe to 3
    dictReplace = {'mild':1, 'moderate':2, 'severe':3}
    dfEchoLabeled['function'] = dfEchoLabeled['function'].fillna(0)
    dfEchoLabeled['function'] = dfEchoLabeled['function'].replace(dictReplace)

    dfEchoLabeled['dilation'] = dfEchoLabeled['dilation'].fillna(0)
    dfEchoLabeled['dilation'] = dfEchoLabeled['dilation'].replace(dictReplace)
    
    return dfEchoLabeled


def merge_labeled_echo(dfMerged, dfEchoLabeled):
    """
    Parse labeled echo data to keep matching enc, order_proc_id
    If multiple rows per enc and order id, keep max bc this 
    represents different line of report
    """
    srsIsNullNarrative = dfMerged['NARRATIVE_compiled'].isnull()
    for nEnc in dfMerged.index:
        nOrder = dfMerged.at[nEnc, 'ORDER_PROC_ID']
        dfTemp = dfEchoLabeled[dfEchoLabeled['HSP_ENC']==nEnc]
        dfTemp = dfTemp[dfTemp['ORDER_PROC_ID']==nOrder]
        if dfTemp.shape[0] > 0:    
            dfMerged.at[nEnc, 'echo_dilation'] = dfTemp['dilation'].max()
            dfMerged.at[nEnc, 'echo_function'] = dfTemp['function'].max()
        elif not srsIsNullNarrative[nEnc]:
            dfMerged.at[nEnc, 'echo_dilation'] = 0
            dfMerged.at[nEnc, 'echo_function'] = 0
    
    return dfMerged

In [ ]:
# Add more features to the verctors

def vect_with_added_features(vect, X_train, X_test, df_features_to_add):
    '''
    add features to the vecterized X_train and X_test
    '''
    def add_feature(X, feature_to_add):
        """
        Returns sparse feature matrix with added feature.
        feature_to_add can also be a list of features.
        """
        from scipy.sparse import csr_matrix, hstack
        return hstack([X, csr_matrix(feature_to_add).T], 'csr')

    
    if type(df_features_to_add) == pd.Series:
        column_names = df_features_to_add.name
    elif type(df_features_to_add) == pd.DataFrame:
        column_names = df_features_to_add.columns
        
    
    X_train_vected = vect.transform(X_train)
    X_train_df = X_train.to_frame()
    X_train_df = X_train_df.merge(df_features_to_add, how='left', left_index=True, right_index=True)
    X_train_df = X_train_df.fillna(0)
    X_train_added = add_feature(X_train_vected, np.array(X_train_df[column_names]).T)

    
    X_test_vected = vect.transform(X_test)
    X_test_df = X_test.to_frame()
    X_test_df = X_test_df.merge(df_features_to_add, how='left',  left_index=True, right_index=True)
    X_test_df = X_test_df.fillna(0)
    X_test_added = add_feature(X_test_vected, np.array(X_test_df[column_names]).T)
    
    return X_train_added, X_test_added

In [ ]:
# Add more features to the verctors

import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score

vect = TfidfVectorizer(min_df=5, ngram_range=(3,10)).fit(X_train)


# new features
X_copy = X.copy()
X_copy['length'] = X_copy['CT_Echo'].map(len)
X_copy['length1'] = X_copy['CT_Echo'].map(len)

#df_new_features = X_copy[['length', 'length1']]
df_new_features = df_new_features_echo_tags

X_train_added, X_test_added = vect_with_added_features(vect, X_train, X_test, df_new_features)


m = LogisticRegression().fit(X_train_added, y_train)
y_scores_m = m.decision_function(X_test_added)
y_proba_m = [i[1] for i in m.predict_proba(X_test_added)]

precision, recall, thresholds = precision_recall_curve(y_test, y_proba_m)
fpr, tpr, _ = roc_curve(y_test, y_proba_m)
roc_auc = auc(fpr, tpr)
roc_auc

In [93]:
def merge_echo(out_df, ech_df, enc_df, rad_df):
    # read each dataset
    mod_out_df = out_df.copy()
    mod_out_df.reset_index(level=0, inplace=True)
    mod_ech_df = ech_df.copy()
    mod_ech_df.reset_index(level=0, inplace=True)
    mod_enc_df = enc_df.copy()
    mod_enc_df.reset_index(level=0, inplace=True)
    mod_rad_df = rad_df.copy()
    mod_rad_df.reset_index(level=0, inplace=True)
    
    df_enc_echo_outcome = get_merge_dfs(mod_out_df, mod_ech_df, mod_enc_df)
    df_enc_CT = merge_encounter_radiology(mod_enc_df, mod_rad_df)

    # clean the dataset
    df_cleaned = df_enc_echo_outcome.merge(df_enc_CT, how='left', on='HSP_ENC')

    # raw column names
    name_gen = []
    for i in df_cleaned.columns.values:
        if i.startswith('name_gen') or i.startswith('NARR'):
            name_gen.append(i)
    name_gen = sorted(name_gen) + ['b48hr']

    # rename the column names
    df_cleaned = df_cleaned[name_gen]
    new_columns = ['CT', 'Echo', 'CPR', 'DEATH', 'INTUBATION', 'PPV', 'THROMBOLYSIS', 'THROMBOLYSIS_PROC', 'VASOPRESSORS', 'B48hr']
    df_cleaned.columns = new_columns

    # fill Nan
    df_cleaned['CT'].fillna('No_CT', inplace=True)
    df_cleaned['Echo'].fillna('No_Echo', inplace=True)
    df_cleaned.fillna(0, inplace=True)

    # convert to 1
    map_dict = {'DEATH': 1, 
                'INTUBATION': 1,
                'PPV': 1,
                'THROMBOLYSIS': 1,
                'THROMBOLYSIS_PROC': 1,
                'VASOPRESSORS': 1,
                0:0}
    for col in ['DEATH', 'INTUBATION', 'PPV', 'THROMBOLYSIS', 'THROMBOLYSIS_PROC', 'VASOPRESSORS']:
        df_cleaned[col] = df_cleaned[col].map(map_dict)
        
    
    echo_tag_path = os.path.join(data_path, 'echo_tag.csv')
    df_echo_label = pd.read_csv(echo_tag_path, encoding='ISO-8859-1')

    dfEchoLabeled = preproc_labeled_echo(df_echo_label)
    dfMerged = merge_labeled_echo(df_enc_echo_outcome, dfEchoLabeled)

    #dfMerged.head()
    df_new_features_echo_tags = dfMerged[['echo_dilation','echo_function']]
    mergeRes = pd.merge(df_cleaned, df_new_features_echo_tags, on='HSP_ENC', how='left')
        
    return mergeRes

## Dropping unused rows

In [131]:
def drop_rows(enc_df, data_path):
    drop_path = os.path.join(data_path, "final_enc_list.csv")
    drop_data = pd.read_csv(drop_path, encoding='ISO-8859-1')
    
    mergeRes = pd.merge(enc_df, drop_data, on='HSP_ENC')
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

##  
## Start calling merge functions

In [106]:
output_data = encounter_df

In [107]:
output_data = merge_echo(outcome_df, echo_df, encounter_df, radiology_df)

In [108]:
output_data = merge_cancer(output_data, cancer_df)

In [109]:
output_data = merge_registry(output_data, registry_df)

In [110]:
output_data = merge_vitals(output_data, data_path)

In [111]:
output_data = merge_co_morbid(output_data, data_path)

In [112]:
output_data = merge_labs(output_data, data_path)

In [113]:
outcome = output_data.pop('B48hr')
output_data['B48hr']=outcome

In [132]:
output_data = drop_rows(output_data, '')

In [119]:
print(output_data.head(25).to_csv("test.csv"))

None


##  
## Preprocess / Normalize the merged data

In [127]:
y = output_data.iloc[:, -1]
X = output_data.copy()
X = X.drop('B48hr', 1)

##  
##  Define params and run gridsearch to find best params

In [ ]:
params = {
    'n_estimators': [5, 10, 15],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, None],
    'min_samples_split': [2, 4]
}

In [ ]:
clf = GridSearchCV(RandomForestClassifier(), params, cv=5, scoring = 'roc_auc')
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print(clf.best_params_)
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))